# [Bert-base-Chinese 微调](https://blog.csdn.net/qq_43668800/article/details/131921617)

In [1]:
import torch
from torch.optim import AdamW
from datasets import load_dataset
from transformers import BertModel, BertTokenizer

## 模型

In [2]:
# 优先使用 GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device=', device)

device= cuda


In [3]:
# 加载预训练模型
pretrained = BertModel.from_pretrained('bert-base-chinese')
# 需要移动到cuda上
pretrained.to(device)

# 不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# 定义下游任务模型
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                             attention_mask=attention_mask,
                             token_type_ids=token_type_ids)
        out = self.fc(out.last_hidden_state[:, 0])
        out = out.softmax(dim=1)
        return out


model = Model()
# 同样要移动到cuda
model.to(device)

Model(
  (fc): Linear(in_features=768, out_features=2, bias=True)
)

## 数据集

In [5]:
# 定义数据集
class Dataset(torch.utils.data.Dataset):

    def __init__(self, split):
        self.dataset = load_dataset('lansinuote/ChnSentiCorp')[split]

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset[i]['text']
        label = self.dataset[i]['label']
        return text, label


dataset = Dataset('train')

Using custom data configuration lansinuote--ChnSentiCorp-4d058ef86e3db8d5
Reusing dataset parquet (/root/.cache/huggingface/datasets/lansinuote___parquet/lansinuote--ChnSentiCorp-4d058ef86e3db8d5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
# 加载字典和分词工具
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [7]:
def collate_fn(data):
    sents = [i[0] for i in data]
    labels = [i[1] for i in data]

    # 编码
    data = tokenizer.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=500,
                                   return_tensors='pt',
                                   return_length=True)
    # input_ids:编码之后的数字
    # attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)
    token_type_ids = data['token_type_ids'].to(device)
    labels = torch.LongTensor(labels).to(device)
    #print(data['length'], data['length'].max())
    return input_ids, attention_mask, token_type_ids, labels


# 数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)
for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
    break

# print(len(loader))
# print(input_ids.shape, attention_mask.shape, token_type_ids.shape, labels)

## 训练

In [8]:
import time, datetime

start_time = datetime.datetime.now()
print('Start time:', start_time)


# 训练
optimizer = AdamW(model.parameters(), lr=5e-4)
criterion = torch.nn.CrossEntropyLoss()
model.train()
for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):
    out = model(input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if i % 5 == 0:
        out = out.argmax(dim=1)
        accuracy = (out == labels).sum().item() / len(labels)
        print(i, loss.item(), accuracy)
    if i == 300:
        break


end_time = datetime.datetime.now()
print('End time:', end_time)

consume_time = end_time - start_time
print('Consume time of second:', consume_time.seconds)

Start time: 2024-03-14 13:52:29.419518
0 0.6470381021499634 0.625
5 0.7437343001365662 0.375
10 0.6767182946205139 0.5625
15 0.6169332265853882 0.8125
20 0.5803598165512085 0.875
25 0.6000330448150635 0.75
30 0.5423067808151245 0.875
35 0.5982635021209717 0.8125
40 0.6188817024230957 0.75
45 0.5755255818367004 0.8125
50 0.5129714012145996 0.9375
55 0.5528658628463745 0.8125
60 0.5478427410125732 0.8125
65 0.5928286910057068 0.6875
70 0.5078943967819214 0.9375
75 0.47246354818344116 0.9375
80 0.6064415574073792 0.75
85 0.4617560803890228 1.0
90 0.49454641342163086 0.875
95 0.5279218554496765 0.75
100 0.5031396746635437 0.875
105 0.4929860234260559 0.875
110 0.5481787919998169 0.8125
115 0.5098983645439148 0.8125
120 0.47813352942466736 0.875
125 0.4316408038139343 0.875
130 0.4767877459526062 0.875
135 0.39120712876319885 1.0
140 0.3908929228782654 1.0
145 0.4696404039859772 0.9375
150 0.480920672416687 0.875
155 0.41305437684059143 0.875
160 0.5231927037239075 0.875
165 0.4650673866271

## 测试

In [9]:
def test():
    model.eval()
    correct = 0
    total = 0
    loader_test = torch.utils.data.DataLoader(dataset=Dataset('validation'),
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)
    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader_test):
        if i%5 == 0 and total != 0:
            print(i, correct / total)
        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)
        out = out.argmax(dim=1)
        correct += (out == labels).sum().item()
        total += len(labels)
    print('Test finish:', correct / total)


test()

Using custom data configuration lansinuote--ChnSentiCorp-4d058ef86e3db8d5
Reusing dataset parquet (/root/.cache/huggingface/datasets/lansinuote___parquet/lansinuote--ChnSentiCorp-4d058ef86e3db8d5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

5 0.85625
10 0.890625
15 0.8708333333333333
20 0.8640625
25 0.865
30 0.8708333333333333
35 0.8741071428571429
Test finish: 0.8741554054054054


## 保存模型

In [10]:
import torch

model_path = '/root/workspace/model/model_bert-base-chinese/emotion_classification.pt'
# torch.save(model.state_dict(), model_path)
torch.save(model.state_dict(), model_path)
print('模型已保存至:', model_path)

模型已保存至: /root/workspace/model/model_bert-base-chinese/emotion_classification.pt


## 加载模型

In [11]:
my_model = Model()
my_model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [12]:
def predict(the_model, text_arr):
    the_model.eval()
    # 优先使用 GPU
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print('device=', device)
    the_model.to(device)

    tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
    torch.no_grad()
    x_encode = tokenizer.batch_encode_plus(
        # 传入的所有句子，有成对句子
        batch_text_or_text_pairs=text_arr,
        # 长度大于设置是否截断
        truncation=True,
        # 一律补齐，如果长度不够
        padding='max_length',
        add_special_tokens=True,
        max_length=30,
        # 可取值tf,pt,np,（tensorflow,pytorch,numpy）默认返回list
        return_tensors="pt",
        # 返回token_type_ids,第一句与特殊符号是0，第二句是1
        return_token_type_ids=True,
        # 返回attention_mask，填充是0，其他是1
        return_attention_mask=True,
        # 返回special_tokens_mask特殊符号标识，特殊是1，其他是0
        return_special_tokens_mask=True,
        # 返回长度,这里的长度是真实长度，而非设置的长度30了
        return_length=True
    )

    y = the_model(input_ids=x_encode['input_ids'].to(device),
                  attention_mask=x_encode['attention_mask'].to(device),
                  token_type_ids=x_encode['token_type_ids'].to(device))
    # print(y)
    res = y.argmax(dim=1)
    return res.cpu().numpy().tolist()


texts = ['这家店不错，下次还来', '唉，有点失望，卫生如果能更好一些就不错了', '还行，跟网上描述的一致，有无线网，网速很快']
predict(my_model, texts)

device= cuda


[1, 0, 1]